In [1]:
import re

import numpy as np
import pandas as pd
import altair as alt

In [67]:
state_pop_1790_1860 = pd.read_csv("../data/state_populations_1790_1860.csv")
state_pop_1870_1950 = pd.read_csv("../data/state_populations_1870_1950.csv")
state_pop_1960_2010 = pd.read_csv("../data/state_populations_1960_2010.csv")

In [68]:
for df in [state_pop_1790_1860, state_pop_1870_1950, state_pop_1960_2010]:
    df["Name"] = df["Name"].str.replace(" *\[note \d+\]","")

In [69]:
state_admitted_year_dict = state_pop_1790_1860.set_index("Name")["Admitted"].to_dict()

In [70]:
state_pop_1790_1860 = state_pop_1790_1860.drop("Admitted", axis=1)

In [71]:
pop_df = state_pop_1790_1860.merge(
    state_pop_1870_1950,
    left_on = "Name", right_on = "Name",
    how = "inner"
)

In [72]:
pop_df = pop_df.merge(
    state_pop_1960_2010,
    left_on = "Name", right_on = "Name",
    how = "inner"
)

In [73]:
state_pop_projected = pd.read_csv("../data/state_population_projections.csv")

In [74]:
state_pop_projected = state_pop_projected.drop(["FIPS", "2010"], axis=1)\
.rename({"Georgaphy Name": "Name"}, axis=1)\
.query("Name.notnull() and Name != 'United States'")

In [75]:
pop_df = pop_df.merge(
    state_pop_projected,
    left_on = "Name", right_on = "Name",
    how = "inner"
)

In [76]:
pop_df = pop_df.set_index("Name")

In [77]:
# add 1 (e.g. 1960 becomes 1961) because decennial census results do not apply in the year they are tabulated
# this makes the forward fill apply 1960 results to 1964, and 1970 results to 1970
pop_df.columns = [int(x)+1 for x in pop_df.columns]

In [78]:
election_years = list(range(1792, 2041, 4))
for col in election_years:
    if col not in pop_df.columns:
        pop_df[col] = np.NaN
for col in pop_df.columns:
    if pop_df[col].dtype == object:
        pop_df[col] = pop_df[col].str.replace(",","").str.replace(" *\[note \d+\]","").astype(float)

In [79]:
pop_df = pop_df[sorted(list(pop_df.columns))]

In [80]:
pop_df = pop_df.fillna(method = "ffill", axis=1)

In [81]:
pop_df = pop_df[election_years]

In [82]:
pop_df = pop_df.reset_index().melt(id_vars = "Name", var_name = "Year", value_name = "Population").dropna()

In [83]:
pop_df["Population"] = pop_df["Population"].astype("int")

In [84]:
alt.Chart(
    pop_df
).mark_line().encode(
    x = "Year",
    y = "Population",
    color = "Name",
    tooltip = ["Name", "Year", "Population"]
)

alt.Chart(...)

In [87]:
elect_df = pd.read_csv("../data/electoral_college_apportionment.csv")

In [88]:
elect_df = elect_df.drop("Number", axis=1)

In [89]:
elect_df["State"] = elect_df["State"].replace("D.C.", "District of Columbia")

In [90]:
elect_df = elect_df.set_index("State")

In [91]:
for col in list(elect_df.columns):
    if "+" in col:
        for new_col in col.split("+"):
            elect_df[int(new_col)] = elect_df[col]
        elect_df = elect_df.drop(col, axis=1)

In [92]:
elect_df.columns = [int(x) for x in elect_df.columns]
elect_df = elect_df[sorted(elect_df.columns)]

In [93]:
elect_df = elect_df.reset_index().melt(id_vars = "State", var_name = "Year", value_name = "Electors")

In [207]:
joined_df = pop_df.merge(
    elect_df,
    left_on = ["Name", "Year"], right_on = ["State", "Year"],
    how = "left"
)

In [208]:
joined_df = joined_df.drop("State", axis=1).rename({"Name":"State"}, axis=1)

In [209]:
joined_df = joined_df.query("Electors.notnull() and Electors > 0").copy()

In [210]:
alt.Chart(joined_df).mark_point().encode(
    x = "Population",
    y = "Electors",
    color = "Year"
)

alt.Chart(...)

In [211]:
joined_df["Residents_per_elector"] = joined_df["Population"]/joined_df["Electors"]

In [212]:
joined_df.query("Year == 1960").sort_values("Residents_per_elector")

,State,Year,Population,Electors,Residents_per_elector
1703,Alaska,1960,128643,3.0,42881.000000
1730,Nevada,1960,160083,3.0,53361.000000
1752,Wyoming,1960,290529,3.0,96843.000000
1709,Delaware,1960,318085,3.0,106028.333333
1747,Vermont,1960,377747,3.0,125915.666667
1731,New Hampshire,1960,533242,4.0,133310.500000
1714,Idaho,1960,588637,4.0,147159.250000
1728,Montana,1960,591024,4.0,147756.000000
1736,North Dakota,1960,619636,4.0,154909.000000
1743,South Dakota,1960,652740,4.0,163185.000000


In [213]:
alt.Chart(joined_df.query("Year > 1900 and Year < 2020")).mark_bar().encode(
    x = alt.X("Residents_per_elector", bin = alt.Bin(maxbins = 20)),
    y = "count()",
    row = "Year"
).properties(height=50)

alt.Chart(...)

In [214]:
alt.Chart(joined_df.query("Year > 1900 and Year < 2020")).transform_density(
    "Residents_per_elector",
    as_ = ["Residents_per_elector", "density"],
    groupby = ["Year"],
    #steps = 5000
).mark_area(
    interpolate='monotone',
    fillOpacity=0.8,
    stroke='lightgray',
    strokeWidth=0.5
).encode(
    x = alt.X("Residents_per_elector", bin=False),
    y = alt.Y("density:Q", axis=None),
    row = "Year"
).properties(
    height = 50
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
).configure_title(
    anchor='end'
)

alt.Chart(...)

In [215]:
yearly_stats_df = joined_df.groupby("Year")["Residents_per_elector"].agg(["min", "max", "mean", "std"]).reset_index()
yearly_stats_df["std_over_mean"] = yearly_stats_df["std"]/yearly_stats_df["mean"]
yearly_stats_df["max_over_min"] = yearly_stats_df["max"]/yearly_stats_df["min"]

In [216]:
joined_df["Max_res_per_elector_this_year"] = joined_df.groupby("Year")["Residents_per_elector"].transform("max")
joined_df["States_this_year"] = joined_df.groupby("Year")["State"].transform("count")
joined_df["Under_res_per_elector_threshold"] = np.where(
    joined_df["Residents_per_elector"] <= (joined_df["Max_res_per_elector_this_year"]/2),
    True,
    False
)
year_threshold_df = (100.*joined_df.groupby("Year")["Under_res_per_elector_threshold"].sum() / joined_df.groupby("Year")["State"].count())\
.to_frame().rename({0:"Percent_states_under_50%_residents_per_elector_threshold"}, axis=1)
yearly_stats_df = yearly_stats_df.merge(
    year_threshold_df,
    left_on = "Year", right_index=True,
    how = "inner"
)

In [217]:
alt.Chart(
    yearly_stats_df
).mark_line().encode(
    y = "std_over_mean",
    x = "Year"
)

alt.Chart(...)

In [218]:
alt.Chart(
    yearly_stats_df
).mark_line().encode(
    y = "max_over_min",
    x = "Year"
)

alt.Chart(...)

In [219]:
alt.Chart(
    yearly_stats_df
).mark_line().encode(
    y = "Percent_states_under_50%_residents_per_elector_threshold",
    x = "Year"
)

alt.Chart(...)

## State Election Results

In [220]:
state_election_results_df = pd.read_csv("../data/state_election_results.csv", index_col=False)

In [221]:
state_election_results_df.shape

(51, 65)

In [222]:
state_election_results_df.head()

,State,1789,1792,1796,1800,Unnamed: 5,1804,1808,1812,1816,...,1984,1988,1992,1996,2000,Unnamed: 60,2004,2008,2012,2016
0,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,R,R,R,R,R,NaN,R,R,R,R
1,Alaska,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,R,R,R,R,R,NaN,R,R,R,R
2,Arizona,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,R,R,R,D,R,NaN,R,R,R,R
3,Arkansas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,R,R,D,D,R,NaN,R,R,R,R
4,California,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,R,R,D,D,D,NaN,D,D,D,D


In [223]:
state_election_results_df = state_election_results_df.drop([x for x in state_election_results_df.columns if "Unnamed" in x], axis=1)

In [224]:
state_election_results_df["State"] = state_election_results_df["State"].replace({"D.C.":"District of Columbia"})

In [225]:
state_election_results_df = state_election_results_df.melt(id_vars = "State", var_name = "Year", value_name = "Party")

In [226]:
state_election_results_df = state_election_results_df.query("Party.notnull()").copy()

In [227]:
state_election_results_df["Year"] = state_election_results_df["Year"].astype(int)

In [228]:
state_election_results_df = state_election_results_df.query("Year > 1865")

In [229]:
state_election_results_df["Party"].value_counts()

R     968
D     797
PR      7
AI      5
SR      4
PO      4
I       2
SP      1
Name: Party, dtype: int64

In [230]:
state_election_results_df["Party"] = state_election_results_df["Party"].replace({"R":"Republican", "D":"Democratic"})

In [231]:
state_election_results_df["Party"] = np.where(
    state_election_results_df["Party"].isin(["Democratic", "Republican"]),
    state_election_results_df["Party"],
    "Other"
)

In [232]:
joined_df = joined_df.merge(
    state_election_results_df,
    left_on = ["State", "Year"], right_on = ["State", "Year"],
    how = "left"
)

In [246]:
alt.Chart(joined_df.query("Year > 2000 and Year < 2020 and Party != 'Other'")).transform_density(
    "Residents_per_elector",
    as_ = ["Residents_per_elector", "density"],
    groupby = ["Year", "Party"],
).mark_area(
    interpolate='monotone',
    fillOpacity=0.8,
    stroke='lightgray',
    strokeWidth=0.5,
    opacity = 0.3
).encode(
    x = alt.X("Residents_per_elector", bin=False),
    y = alt.Y("density:Q", axis=None),
    color = "Party",
    row = "Year"
).properties(
    height = 100
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
).configure_title(
    anchor='end'
)

alt.Chart(...)

In [245]:
alt.Chart(joined_df.query("Year > 2000 and Year < 2020 and Party != 'Other'")).mark_bar(
    opacity = 0.3
).encode(
    x = alt.X("Residents_per_elector", bin=True),
    y = alt.Y("count()", axis=None, stack=None),
    color = "Party",
    row = "Year"
).properties(
    height = 100
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
).configure_title(
    anchor='end'
)

alt.Chart(...)

In [242]:
yearly_party_stats_df = joined_df.groupby(["Year", "Party"])["Residents_per_elector"].agg(["min", "max", "mean", "std"]).reset_index()
yearly_party_stats_df["std_over_mean"] = yearly_party_stats_df["std"]/yearly_party_stats_df["mean"]
yearly_party_stats_df["max_over_min"] = yearly_party_stats_df["max"]/yearly_party_stats_df["min"]

In [249]:
alt.Chart(
    yearly_party_stats_df.query("Party != 'Other'")
).mark_line().encode(
    y = "max_over_min",
    x = "Year",
    color = "Party"
)

alt.Chart(...)